In [1]:
!pip install langchain

In [2]:
import langchain

In [3]:
#sets your API key into the environment variables that Python can read
import os
os.environ["OPENAI_API_KEY"]="***" 

In [4]:
!pip install openai


In [5]:
# check 
from openai import OpenAI
client = OpenAI()   #Creates a client object that will handle requests to OpenAI’s servers.“connection” ready to talk to OpenAI’s API.

response = client.chat.completions.create(  #sends a request to OpenAI’s Chat Completions API.
    model="gpt-4o-mini",
    messages=[{"role": "user", "content": "Hello, AI!"}]
)

print(response.choices[0].message.content)


Hello! How can I assist you today?


In [6]:
os.environ["LANGCHAIN_TRACING_V2"]="true" # tracing
os.environ["LANGCHAIN_API_KEY"]="***"  # langsmith for tracing and monitoring
os.environ["LANGCHAIN_PROJECT"]="langchain-course"   # project name

# CALL LLM inside langchain

In [7]:
!pip install langchain-openai


In [8]:
    from langchain_openai import ChatOpenAI #Imports the LangChain wrapper for OpenAI’s chat models.

    llm = ChatOpenAI(model="gpt-4o-mini")
    llm_response = llm.invoke("Tell me a joke")
    print(llm_response)


content='Why don’t skeletons fight each other? \n\nThey don’t have the guts!' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 11, 'total_tokens': 28, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_560af6e559', 'id': 'chatcmpl-CMxvIw16drX296DoLZXAYKvxMkaX3', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None} id='run--73077385-7220-4a08-ba03-9829cb3e9ddd-0' usage_metadata={'input_tokens': 11, 'output_tokens': 17, 'total_tokens': 28, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}


# Output Parsers

In [9]:
    from langchain_core.output_parsers import StrOutputParser
#Imports StrOutputParser, a parser that takes the raw model output (which is usually a structured object) and converts it into a simple string.

    output_parser = StrOutputParser()
    chain = llm | output_parser   # chain pass o/p of llm to outputparser
    result = chain.invoke("Tell me a joke")
    print(result)


Why don't scientists trust atoms? 

Because they make up everything!


# Structured output
    

In [10]:
# LLM to analyze the review and give the structure output

In [11]:
from typing import List
from pydantic import BaseModel, Field

class MobileReview(BaseModel):
        phone_model: str = Field(description="Name and model of the phone")
        rating: float = Field(description="Overall rating out of 5")
        pros: List[str] = Field(description="List of positive aspects")
        cons: List[str] = Field(description="List of negative aspects")
        summary: str = Field(description="Brief summary of the review")

review_text = """
    Just got my hands on the new Galaxy S21 and wow, this thing is slick! The screen is gorgeous,
    colors pop like crazy. Camera's insane too, especially at night - my Insta game's never been
    stronger. Battery life's solid, lasts me all day no problem.
    Not gonna lie though, it's pretty pricey. And what's with ditching the charger? C'mon Samsung.
    Also, still getting used to the new button layout, keep hitting Bixby by mistake.
    Overall, I'd say it's a solid 4 out of 5. Great phone, but a few annoying quirks keep it from
    being perfect. If you're due for an upgrade, definitely worth checking out!
    """
structured_llm = llm.with_structured_output(MobileReview)
output = structured_llm.invoke(review_text)
print(output)
print(output.pros)


phone_model='Samsung Galaxy S21' rating=4.0 pros=['Gorgeous screen with vibrant colors', 'Insane camera quality, especially at night', 'Solid battery life, lasts all day'] cons=['Pricey compared to competitors', 'No charger included in the box', 'New button layout can be confusing, accidental Bixby activations'] summary='The Galaxy S21 is a fantastic smartphone with an amazing display and impressive camera capabilities, but its high price and some design choices might annoy a few users.'
['Gorgeous screen with vibrant colors', 'Insane camera quality, especially at night', 'Solid battery life, lasts all day']


# Prompt Templates

In [12]:
    from langchain_core.prompts import ChatPromptTemplate

    prompt = ChatPromptTemplate.from_template("Tell me a short joke about {topic}")
    chain = prompt | llm | output_parser
    result = chain.invoke({"topic": "programming"})
    print(result)


Why do programmers prefer dark mode?

Because light attracts bugs!


In [13]:
result=chain.invoke({"topic":"car drivers"})
print(result)

Why did the car driver break up with their GPS? 

Because they just couldn't handle the constant rerouting!


In [14]:
# Define the prompt
# Initialize the LLM
# Define the output parser
# Compose the chain
#Use the chain

# LLM Messages

In [15]:
    from langchain_core.messages import HumanMessage, SystemMessage

    messages = [
        SystemMessage(content="You are a helpful assistant that tells jokes."),
        HumanMessage(content="Tell me about programming")
    ]
    response = llm.invoke(messages)
    print(response)

    template = ChatPromptTemplate([
        ("system", "You are a helpful assistant that tells jokes."),
        ("human", "Tell me about {topic}")
    ])
    chain = template | llm
    response = chain.invoke({"topic": "programming"})
    print(response)


content='Sure! Here’s a little joke about programming for you:\n\nWhy do programmers prefer dark mode?\n\nBecause light attracts bugs! \n\nIf you have any specific questions about programming or want to hear more jokes, just let me know!' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 45, 'prompt_tokens': 24, 'total_tokens': 69, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_560af6e559', 'id': 'chatcmpl-CMxwP3GUNEAaVsWs55T5z5froKkcW', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None} id='run--d08a89ee-995e-4ee2-9638-451c7322666b-0' usage_metadata={'input_tokens': 24, 'output_tokens': 45, 'total_tokens': 69, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'rea

# Document Processing for RAG Systems

# Loading Documents

In [16]:
!pip install -q langchain


In [17]:
!pip install -q langchain-community langchain-text-splitters


In [18]:
!pip install docx2txt


In [19]:
!pip install pypdf


In [20]:
from langchain_community.document_loaders import PyPDFLoader, Docx2txtLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from typing import List
from langchain_core.documents import Document
import os

def load_documents(folder_path: str) -> List[Document]:
    documents = []
    for filename in os.listdir(folder_path):
        file_path = os.path.join(folder_path, filename)
        if filename.endswith('.pdf'):
            loader = PyPDFLoader(file_path)
        elif filename.endswith('.docx'):
            loader = Docx2txtLoader(file_path)
        else:
            print(f"Unsupported file type: {filename}")
            continue
        documents.extend(loader.load())
    return documents

folder_path = "DS_Assingments/RAG DOC TEST"
documents = load_documents(folder_path)
print(f"Loaded {len(documents)} documents from the folder.")


Unsupported file type: .ipynb_checkpoints
Loaded 5 documents from the folder.


# Splitting Documents

In [21]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, #break down our documents into chunks of about 1000 characters, 
    chunk_overlap=200, #This overlap helps maintain context between chunks.
    length_function=len
)

splits = text_splitter.split_documents(documents)
print(f"Split the documents into {len(splits)} chunks.")


Split the documents into 8 chunks.


In [22]:
print(documents[0])


page_content='Drone Maintenance Guide - Model XR-210

The XR-210 drone is a high-performance device designed for both recreational and professional use. To ensure 
     maximum reliability, owners must follow a strict maintenance schedule. Firmware updates are released quarterly, 
     offering improved flight algorithms and bug fixes. Before every flight, propellers should be inspected for damage, 
     as even small cracks can affect stability. The lithium-polymer batteries have a life cycle of about 500 charges, 
     after which replacements are recommended to avoid sudden failures. For safety, the XR-210 should never be operated 
     in winds stronger than 35 km/h, as turbulence can overwhelm the stabilization system. After any crash, the GPS 
     sensors must be recalibrated to restore accurate navigation. With proper care, the XR-210 can operate reliably for 
     over five years, making it a trusted choice for aerial photography, surveying, and agricultural monitoring.' metad

In [23]:
print(splits[1])


page_content='GreenCart Pvt. Ltd. - Annual Business Review 2024-25
GreenCart has seen remarkable growth during the financial year 2024-25. The company recorded
an 18% increase in 
     revenue compared to the previous fiscal year, largely attributed to the expansion of its
eco-friendly product 
     catalog. With increasing environmental awareness, customers across India and Southeast Asia
showed stronger demand 
     for sustainable alternatives. GreenCart entered Singapore and Malaysia, with both markets
showing double-digit 
     growth. However, supply chain delays - especially in raw materials imported from Europe -
presented significant 
     operational challenges. The leadership team responded by diversifying suppliers and investing in
local 
     manufacturing hubs. Moving forward, GreenCart plans to launch a subscription-based
sustainability program, 
     offering monthly eco-friendly essentials to households. This initiative is expected to deepen
customer loyalty and' metad

In [24]:
print(splits[0].metadata)


{'source': 'DS_Assingments/RAG DOC TEST\\Drone_Maintenance_Guide_Model_XR-210.docx'}


# Creating Embeddings for RAG Systems

In [25]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")
emb = model.encode("This is a test sentence.")
print(len(emb), emb[:5])  # should print 384 and some numbers



384 [0.08429645 0.0579537  0.00449338 0.10582107 0.00708338]


In [26]:
from langchain_community.embeddings import HuggingFaceEmbeddings

# Choose a free model from HuggingFace
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# Generate embeddings for your chunks
document_embeddings = embeddings.embed_documents([split.page_content for split in splits])
print(f"Created embeddings for {len(document_embeddings)} document chunks.")


C:\Users\Prerana\AppData\Local\Temp\ipykernel_7520\2785407333.py:4: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")


Created embeddings for 8 document chunks.


In [27]:
from langchain_community.embeddings.sentence_transformer import SentenceTransformerEmbeddings

embedding_function = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")
document_embeddings = embedding_function.embed_documents([split.page_content for split in splits])
print(document_embeddings[0][:5])  # Printing first 5 elements of the first embedding


[-0.014027595520019531, 0.019634757190942764, 0.03235810250043869, -0.09138241410255432, 0.06517262756824493]


# Setting Up the Vector Store for RAG Systems

In [28]:
# chroma vector store
#Chroma is an open-source vector database that is:

#Free & lightweight (runs locally by default, no setup needed).

#Integrated with LangChain (so you can use it in just a few lines of code).

#Stores your document chunks + embeddings.

#Lets you querypip install chromadb
 

In [29]:
!pip install chromadb


In [30]:
from langchain_community.vectorstores import Chroma


collection_name = "my_collection"
vectorstore = Chroma.from_documents(
    collection_name=collection_name,
    documents=splits,
    embedding=embedding_function,
    persist_directory="./chroma_db"
)
print("Vector store created and persisted to './chroma_db'")


Vector store created and persisted to './chroma_db'


# Performing Similarity Search

In [31]:
query = "What does the document say about AI in healthcare?"
search_results = vectorstore.similarity_search(query, k=2)  #Converts the query → embedding (vector).
print(f"\nTop 2 most relevant chunks for the query: '{query}'\n")
for i, result in enumerate(search_results, 1):
    print(f"Result {i}:")
    print(f"Source: {result.metadata.get('source', 'Unknown')}")
    print(f"Content: {result.page_content}")
    print()



Top 2 most relevant chunks for the query: 'What does the document say about AI in healthcare?'

Result 1:
Source: DS_Assingments/RAG DOC TEST\Meditation_Benefits_for_Professionals.pdf
Content: tied to 
     workplace efficiency, creativity, and decision-making. For organizations, encouraging meditation
is not only an 
     employee wellness initiative but also a long-term investment in productivity and workplace
harmony.

Result 2:
Source: DS_Assingments/RAG DOC TEST\Meditation_Benefits_for_Professionals.pdf
Content: tied to 
     workplace efficiency, creativity, and decision-making. For organizations, encouraging meditation
is not only an 
     employee wellness initiative but also a long-term investment in productivity and workplace
harmony.



# Creating a Retriever

In [32]:
retriever = vectorstore.as_retriever(search_kwargs={"k": 2}) #Converts your Chroma vector store into a retriever object.
retriever_results = retriever.invoke("When was GreenGrow Innovations founded?")
print(retriever_results)


[Document(metadata={'total_pages': 1, 'creator': 'PyPDF', 'creationdate': 'D:20251003092216', 'producer': 'PyFPDF 1.7.2 http://pyfpdf.googlecode.com/', 'page_label': '1', 'source': 'DS_Assingments/RAG DOC TEST\\GreenCart_Annual_Business_Review_2024-25.pdf', 'page': 0}, page_content='GreenCart Pvt. Ltd. - Annual Business Review 2024-25\nGreenCart has seen remarkable growth during the financial year 2024-25. The company recorded\nan 18% increase in \n     revenue compared to the previous fiscal year, largely attributed to the expansion of its\neco-friendly product \n     catalog. With increasing environmental awareness, customers across India and Southeast Asia\nshowed stronger demand \n     for sustainable alternatives. GreenCart entered Singapore and Malaysia, with both markets\nshowing double-digit \n     growth. However, supply chain delays - especially in raw materials imported from Europe -\npresented significant \n     operational challenges. The leadership team responded by diver

# Creating the RAG Chain

In [33]:
from langchain.schema.runnable import RunnablePassthrough
template = """Answer the question based only on the following context:
{context}
Question: {question}
Answer: """

prompt = ChatPromptTemplate.from_template(template)

def docs2str(docs):
    return "\n\n".join(doc.page_content for doc in docs)

rag_chain = (
    {"context": retriever | docs2str, "question": RunnablePassthrough()} #don’t transform it, just pass it along unchanged)
    | prompt
    | llm
    | StrOutputParser()
)

In [34]:
question = "How much daily mindfulness practice is recommended for improving resilience and job satisfaction?"
response = rag_chain.invoke(question)
print(f"Question: {question}")
print(f"Answer: {response}")


Question: How much daily mindfulness practice is recommended for improving resilience and job satisfaction?
Answer: Just 10 minutes of mindfulness each day is recommended for improving resilience and job satisfaction.


# Handling Follow-Up Questions

# Creating a history aware retriever

In [35]:
from langchain_core.messages import HumanMessage,AIMessage
chat_history=[]
chat_history.extend([HumanMessage(content=question),
                     AIMessage(content=response)
                    ])

In [36]:
chat_history

[HumanMessage(content='How much daily mindfulness practice is recommended for improving resilience and job satisfaction?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='Just 10 minutes of mindfulness each day is recommended for improving resilience and job satisfaction.', additional_kwargs={}, response_metadata={})]

In [39]:
from langchain_core.prompts import MessagesPlaceholder
from langchain.chains import create_history_aware_retriever
from langchain.chains.combine_documents import create_stuff_documents_chain

contextualize_q_system_prompt = """
Given a chat history and the latest user question
which might reference context in the chat history,
formulate a standalone question which can be understood
without the chat history. Do NOT answer the question,
just reformulate it if needed and otherwise return it as is.
"""

contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_q_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)

contextualize_chain = contextualize_q_prompt | llm | StrOutputParser()
print(contextualize_chain.invoke({"input": "And how does it impact sleep quality?", "chat_history": chat_history}))


What is the impact of daily mindfulness practice on sleep quality?


In [40]:
from langchain.chains import create_retrieval_chain

history_aware_retriever = create_history_aware_retriever(
    llm, retriever, contextualize_q_prompt
)
# When a user asks a new question:
# It first rewrites it into a standalone question (using contextualize_q_prompt + chat_history).
# Then runs the rewritten query against Chroma to fetch relevant chunks.

qa_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful AI assistant. Use the following context to answer the user's question."),
    ("system", "Context: {context}"), #will be filled with the retrieved document chunks
    MessagesPlaceholder(variable_name="chat_history"),
    ("human", "{input}")
])# Defines how the answering LLM should behave.

question_answer_chain = create_stuff_documents_chain(llm, qa_prompt) #Stuff documents” means all retrieved chunks are stuffed directly into the prompt
rag_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)
# history_aware_retriever (rewrites question + fetches chunks).
# question_answer_chain (inserts context + history + input into LLM).


# Using the History-Aware RAG Chain

In [44]:
chat_history = []
question1 = "How much daily mindfulness practice is recommended for improving resilience and job satisfaction?"
answer1 = rag_chain.invoke({"input": question1, "chat_history": chat_history})['answer']
chat_history.extend([
    HumanMessage(content=question1),
    AIMessage(content=answer1)
])

print(f"Human: {question1}")
print(f"AI: {answer1}\n")

question2 = "And how does it impact sleep quality?"
answer2 = rag_chain.invoke({"input": question2, "chat_history": chat_history})['answer']
chat_history.extend([
    HumanMessage(content=question2),
    AIMessage(content=answer2)
])

print(f"Human: {question2}")
print(f"AI: {answer2}")

Human: How much daily mindfulness practice is recommended for improving resilience and job satisfaction?
AI: Scientific studies confirm that just 10 minutes of mindfulness practice each day can improve overall job satisfaction and resilience.

Human: And how does it impact sleep quality?
AI: Meditation has a positive impact on sleep quality by helping to reduce stress and anxiety, which can often interfere with restful sleep. Improved sleep quality is directly tied to workplace efficiency, creativity, and decision-making, making meditation a valuable practice for professionals aiming to enhance their overall performance.
